In [0]:
import os
import tensorflow
from PIL import ImageFile, Image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import save_img
from keras.preprocessing.image import img_to_array
import glob
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, BatchNormalization, LSTM, TimeDistributed, SpatialDropout1D
from google.colab.patches import cv2_imshow

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# image size
img_width, img_height = 148, 148
# Рgrayscale image 148x148
# backend Tensorflow => channels_last
input_shape = (img_width, img_height, 1)

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [21]:
# filling object-feature matrix
X = []
filenames = []
c = 0
for filename in sorted(glob.glob('/content/drive/My Drive/train/*.jpg')):
  image = cv2.imread(filename)
  image = cv2.resize(image, (img_width, img_height))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  filenames.append(filename)
  X.append(image)
  c += 1
  print(c)
print(X[0])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
# for filename in sorted(glob.glob('/content/drive/My Drive/train/*.jpg')):
#   image = cv2.imread(filename)
#   image = cv2.resize(image, (img_width, img_height))
#   image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#   print(cv2_imshow(image))

In [0]:
# setting pixels values to range [0;1]
X = np.divide(X, 255.0)

In [0]:
# reading file with answers for images 
Y = pd.read_csv('/content/drive/My Drive/training.csv', sep=',')
Y = Y[['subDirectory_filePath', 'valence', 'arousal']]
Y['subDirectory_filePath'] = Y['subDirectory_filePath'].apply(lambda x: x.split("/")[1])
Y = Y.sort_values('subDirectory_filePath')

In [24]:
Y.head()

,subDirectory_filePath,valence,arousal
34738,0000792211b64f6a59bd2d95fee49eabe6373ec1d88f22...,-0.488145,0.821877
157036,0000f8a4575c15055a9ee0a72c9aa5bf9ac00558173565...,-0.316056,-0.136957
26740,00012312112a15995f1d1c1ac640db7191eacab8099e90...,0.710109,0.689223
178824,0001636b7a16a63b2d9c8f5e4b7be02e4841d3e0af3ebd...,-0.420635,0.515873
298470,0001720743cf22095bd3b2c94b35f244faf47545e26168...,0.521274,0.126796


In [0]:
# Y1 = []
# for index, row in Y.iterrows():
#   for f in filenames:
#     if f.find(row['subDirectory_filePath']) >= 0:
#       Y1.append(row)
#       print("appending " + str(row))
#       break

In [25]:
df = pd.read_csv('/content/drive/My Drive/y_modified.csv', sep=',')
df = df[['subDirectory_filePath', 'valence', 'arousal']]
df

,subDirectory_filePath,valence,arousal
0,000c10a8f6735212d71eb54298bc47ea56dc08e64ed2ca...,0.004313,-0.008627
1,001fffed9707757082906ff2eb89cad6ddf242195fe6a5...,0.539683,0.023810
2,002b6db57713be4a264d8dd13ee543b031de7577af983c...,-0.193573,0.590399
3,003e5febbe7808b02f6cafe17878cdfd59851726451209...,0.526805,-0.036153
4,004b10f6bfbce7f58d124ac653451bdd975a7113d20b18...,0.849206,0.238095
...,...,...,...
2137,7a2b38adf6aa2d7b8b93d02f5defb7a6aebd738f8e544e...,0.380952,0.063492
2138,7a2e5d157c9cbe119103f1a60052350ef3cbd14f651628...,-0.111111,-0.920635
2139,7a7b4f6fb23aaa9189a6faf644d07780947b00048d6992...,0.552105,0.219979
2140,7a7ea6d03d1cf4275e6dfc6d26104467c079edccecfcb0...,0.711698,0.125086


In [26]:
# concatenation filenames with X matrix
d = dict(zip(filenames, X))
list(d.keys())[0]

'/content/drive/My Drive/train/000c10a8f6735212d71eb54298bc47ea56dc08e64ed2cab8913d683f.jpg'

In [27]:
files = df['subDirectory_filePath']
new_dict = {}
for key in d.keys():
  for file in files:
    if key.find(file) >= 0:
      new_dict.update({key:d[key]})
      break
print(len(new_dict))

2142


In [0]:
df['image'] = list(new_dict.values())

In [0]:
indexes_to_drop = df[(df['valence'] == -2) | (df['arousal'] == -2)].index
df.drop(indexes_to_drop, inplace=True)

In [50]:
# common model for both valence and arousal
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

model.summary()

model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_57 (Conv2D)           (None, 148, 148, 32)      320       
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 146, 146, 32)      9248      
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 69, 69, 64)        36928     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 34, 34, 64)      

In [46]:
# valence training
x_train_v, x_test_v, y_train_v, y_test_v = train_test_split(np.stack(list(df['image']), axis=0), list(df['valence']), test_size=0.33)

x_train_v = x_train_v.reshape(x_train_v.shape[0], 150, 150, 1)
x_test_v = x_test_v.reshape(x_test_v.shape[0], 150, 150, 1)

model_v.fit(x_train_v, y_train_v, epochs=100, batch_size=64, validation_data=(x_test_v, y_test_v))



Train on 1179 samples, validate on 581 samples
Epoch 1/100
1179/1179 [==============================] - 5s 4ms/step - loss: 0.2473 - acc: 0.0263 - val_loss: 0.2505 - val_acc: 0.0275
Epoch 2/100
1179/1179 [==============================] - 1s 1ms/step - loss: 0.2396 - acc: 0.0263 - val_loss: 0.2568 - val_acc: 0.0275
Epoch 3/100
1179/1179 [==============================] - 1s 1ms/step - loss: 0.2363 - acc: 0.0263 - val_loss: 0.2556 - val_acc: 0.0275
Epoch 4/100
1179/1179 [==============================] - 1s 1ms/step - loss: 0.2336 - acc: 0.0263 - val_loss: 0.2696 - val_acc: 0.0275
Epoch 5/100
1179/1179 [==============================] - 1s 1ms/step - loss: 0.2326 - acc: 0.0263 - val_loss: 0.2757 - val_acc: 0.0275
Epoch 6/100
1179/1179 [==============================] - 1s 1ms/step - loss: 0.2365 - acc: 0.0263 - val_loss: 0.2544 - val_acc: 0.0275
Epoch 7/100
1179/1179 [==============================] - 1s 1ms/step - loss: 0.2348 - acc: 0.0263 - val_loss: 0.2609 - val_acc: 0.0275
Epoch 

In [51]:
# arousal training
x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(np.stack(list(df['image']), axis=0), list(df['arousal']), test_size=0.33)

model_a.fit(x_train_a, y_train_a, epochs=100, batch_size=128, validation_data=(x_test_a, y_test_a))

RuntimeError: ignored

In [47]:
print("score = " + str(1 - model_v.evaluate(x_test_v, y_test_v)[0]))

581/581 [==============================] - 0s 594us/step
score = 0.793989528220615


In [0]:
# saving valence model to file
model_json_v = model_v.to_json()

json_file = open("affect_model_v.json", "w")
json_file.write(model_json_v)
json_file.close()

model_v.save_weights("affect_model_v.h5")

In [0]:
# saving arousal model to file
model_json_a = model_a.to_json()

json_file = open("affect_model_a.json", "w")
json_file.write(model_json_a)
json_file.close()

model_a.save_weights("affect_model_a.h5")

In [0]:
# separate script for emotion recognition 
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image
import argparse as ar

parser = ar.ArgumentParser()
parser.add_argument('path', metavar='path', type=str, nargs='+', help='an integer for the accumulator')
img_path = parser.parse_args().path[0]
img = image.load_img(img_path, target_size=(150, 150))

x = image.img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

json_file_v = open("affect_model_v.json", "r")
json_file_a = open("affect_model_a.json", "r")

loaded_model_json_v = json_file_v.read()
loaded_model_json_a = json_file_a.read()

json_file_v.close()
json_file_a.close()

loaded_model_v = model_from_json(loaded_model_json_v)
loaded_model_a = model_from_json(loaded_model_json_a)

loaded_model_v.load_weights("affect_model_v.h5")
loaded_model_a.load_weights("affect_model_a.h5")

loaded_model_v.compile(loss='mean_squared_error', optimizer='sgd')
loaded_model_a.compile(loss='mean_squared_error', optimizer='sgd')

prediction_v = loaded_model_v.predict(x)
prediction_a = loaded_model_a.predict(x)

if prediction_v >= -0.5 and prediction_v <= 0.5 and prediction_a <= -0.66:
    print("focus")
elif prediction_v >= -0.2 and prediction_a >= -0.66 and prediction_a <= 0:
    print("relaxation")
elif prediction_v <= 0 and prediction_a <= 0:
    print("stress")
elif prediction_v >= 0 and prediction_v <= 0.4 and prediction_a >= 0 and prediction_a <= 0.4:
    print("interest")
elif (prediction_v >= 0 and prediction_v <= 0.7 and prediction_a >= 0.4 and prediction_a <= 0.7 or
      prediction_v >= 0.4 and prediction_v <= 0.7 and prediction_a >= 0 and prediction_a <= 0.4 or
      prediction_v >= 0.7 and prediction_v <= 1 and prediction_a >= 0 and prediction_a <= 1):
    print("engagement")
elif prediction_v >= 0 and prediction_v <= 1 and prediction_a >= 0.7 and prediction_a <= 1:
    print("excitement")
else:
    print("Cannot determine emotion")